In [1]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

conn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='omop')

omopcursor = conn.cursor()
print(conn)

<_mysql.connection open to '127.0.0.1' at 0x562cab4fed80>


In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

In [3]:
df_SPECIMEN = pd.read_sql('select * from `SPECIMEN`', con=conn)

/tmp/ipykernel_33601/2350436099.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SPECIMEN = pd.read_sql('select * from `SPECIMEN`', con=conn)


In [4]:
print('Length - SPECIMEN: ', len(df_SPECIMEN))

Length - SPECIMEN:  0


In [5]:
print(df_SPECIMEN.head())

Empty DataFrame
Columns: [specimen_id, person_id, specimen_concept_id, specimen_type_concept_id, specimen_date, specimen_datetime, quantity, unit_concept_id, anatomic_site_concept_id, disease_status_concept_id, specimen_source_id, specimen_source_value, unit_source_value, anatomic_site_source_value, disease_status_source_value]
Index: []


In [6]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

mimicivconn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='mimiciv')

mimicivcursor = conn.cursor()
print(conn)

<_mysql.connection open to '127.0.0.1' at 0x562cab4fed80>


In [ ]:
df_patients_mimiciv = pd.read_sql('select subject_id, anchor_year, anchor_year_group from patients', con=mimicivconn)

In [ ]:
ddf_patients_mimiciv = dd.from_pandas(df_patients_mimiciv, chunksize=100)

In [ ]:
df_patients_mimiciv = ''

In [ ]:
print(ddf_patients_mimiciv)

In [ ]:
df_admissions_mimiciv = pd.read_sql('select subject_id from admissions', con=mimicivconn)

In [ ]:
ddf_admissions_mimiciv = dd.from_pandas(df_admissions_mimiciv, chunksize=100)

In [ ]:
df_admissions_mimiciv = ''

In [ ]:
print(ddf_admissions_mimiciv)

In [ ]:
df_d_labitems_mimiciv = pd.read_sql('select itemid, fluid from d_labitems', con=mimicivconn)

In [ ]:
ddf_d_labitems_mimiciv = dd.from_pandas(df_d_labitems_mimiciv, chunksize=100)

In [ ]:
df_d_labitems_mimiciv = ''

In [ ]:
print(ddf_d_labitems_mimiciv)

In [ ]:
df_labevents_mimiciv = pd.read_sql('select subject_id, specimen_id, itemid, charttime  from labevents', con=mimicivconn)

In [ ]:
ddf_labevents_mimiciv = dd.from_pandas(df_labevents_mimiciv, chunksize=100)

In [ ]:
df_labevents_mimiciv = ''

In [ ]:
print(ddf_labevents_mimiciv)

In [ ]:
ddf_adm_pat_merged = ddf_admissions_mimiciv.merge(ddf_patients_mimiciv, how='inner', on='subject_id')
print(ddf_adm_pat_merged) 

In [ ]:
ddf_lab_merged = ddf_labevents_mimiciv.merge(ddf_d_labitems_mimiciv, how='inner', on='itemid')
print(ddf_lab_merged)

In [ ]:
ddf_lab_adm_merged = ddf_lab_merged.merge(ddf_adm_pat_merged, how='inner', on='subject_id')
print(ddf_lab_adm_merged)

In [ ]:
ddf_lab_adm_merged['quantity'] = np.nan
print(ddf_lab_adm_merged[['quantity']])

In [ ]:
df_lab_adm_merged = pd.read_sql('''SELECT 
            l.subject_id, 
            l.specimen_id, 
            l.itemid, 
            l.charttime, 
            p.anchor_year, 
            p.anchor_year_group, 
            d.fluid
        FROM 
            labevents l, 
            patients p, 
            d_labitems d
        WHERE
            d.itemid = l.itemid''', con=mimicivconn, chunksize=5000)

/tmp/ipykernel_33601/423771016.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_lab_adm_merged = pd.read_sql('''SELECT
